In [ ]:
import sys
import os


if os.path.abspath('..') not in sys.path:
    sys.path.insert(0, os.path.abspath('..'))

In [ ]:
from tqdm.auto import tqdm
import torch
import gpytorch as gp
import altair as alt
import pandas as pd
import numpy as np

from bi_gp.bilateral_kernel import BilateralKernel
from experiments.utils import UCIDataset, standardize

In [ ]:
class ExactGPModel(gp.models.ExactGP):
    def __init__(self, train_x, train_y):
        likelihood = gp.likelihoods.GaussianLikelihood()
        super().__init__(train_x, train_y, likelihood)
        self.mean_module = gp.means.ConstantMean()
        self.covar_module = gp.kernels.ScaleKernel(gp.kernels.RBFKernel(ard_num_dims=train_x.size(-1)))

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gp.distributions.MultivariateNormal(mean_x, covar_x)


class SGPRModel(gp.models.ExactGP):
    def __init__(self, train_x, train_y, inducing_points):
        likelihood = gp.likelihoods.GaussianLikelihood()
        super().__init__(train_x, train_y, likelihood)
        self.mean_module = gp.means.ConstantMean()
        self.base_covar_module = gp.kernels.ScaleKernel(gp.kernels.RBFKernel(ard_num_dims=train_x.size(-1)))
        self.covar_module = gp.kernels.InducingPointKernel(
          self.base_covar_module, inducing_points=inducing_points, likelihood=likelihood)

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gp.distributions.MultivariateNormal(mean_x, covar_x)


class KISSGPModel(gp.models.ExactGP):
    def __init__(self, train_x, train_y, grid_size):
        likelihood = gp.likelihoods.GaussianLikelihood()
        super().__init__(train_x, train_y, likelihood)

        if not isinstance(grid_size, int):
            grid_size = gp.utils.grid.choose_grid_size(train_x)

        self.mean_module = gp.means.ConstantMean()
        self.covar_module = gp.kernels.ScaleKernel(
            gp.kernels.GridInterpolationKernel(
                gp.kernels.RBFKernel(ard_num_dims=train_x.size(-1)), grid_size=grid_size, num_dims=train_x.size(-1)
            )
        )

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gp.distributions.MultivariateNormal(mean_x, covar_x)


class SKIPGPModel(gp.models.ExactGP):
    def __init__(self, train_x, train_y, grid_size):
        likelihood = gp.likelihoods.GaussianLikelihood()
        super().__init__(train_x, train_y, likelihood)
        
        self.mean_module = gp.means.ConstantMean()
        self.base_covar_module = gp.kernels.RBFKernel()
        self.covar_module = gp.kernels.ProductStructureKernel(
            gp.kernels.ScaleKernel(
                gp.kernels.GridInterpolationKernel(self.base_covar_module, grid_size=grid_size, num_dims=1)
            ), num_dims=train_x.size(-1)
        )

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gp.distributions.MultivariateNormal(mean_x, covar_x)


class BilateralGPModel(gp.models.ExactGP):
    def __init__(self, train_x, train_y):
        likelihood = gp.likelihoods.GaussianLikelihood()
        super().__init__(train_x, train_y, likelihood)
        self.mean_module = gp.means.ConstantMean()
        self.covar_module = gp.kernels.ScaleKernel(BilateralKernel(ard_num_dims=train_x.size(-1)))

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gp.distributions.MultivariateNormal(mean_x, covar_x)

In [ ]:
def train(x, y, model, mll, optim):
    model.train()

    optim.zero_grad()

    output = model(x)

    loss = -mll(output, y)

    loss.backward()

    optim.step()

    return { 'train/ll': -loss.detach().item() }


def test(x, y, model, lanc_iter=100, pre_size=0):
    model.eval()

    with torch.no_grad():
#        gp.settings.max_preconditioner_size(pre_size), \
#        gp.settings.max_root_decomposition_size(lanc_iter), \
#        gp.settings.fast_pred_var():
        preds = model(x)

        pred_y = model.likelihood(model(x))
        rmse = (pred_y.mean - y).pow(2).mean(0).sqrt()

    return { 'test/rmse': rmse.item() }


def train_util(model, x, y, lr=0.1, epochs=200):
    mll = gp.mlls.ExactMarginalLogLikelihood(model.likelihood, model)
    optim = torch.optim.Adam(model.parameters(), lr=lr)

    for _ in tqdm(range(epochs)):
        train(x, y, model, mll, optim)

## Pumadyn Dataset

In [ ]:
uci_data_dir = os.path.join(os.environ.get('DATADIR'), 'uci')
train_dataset = UCIDataset.create('pumadyn32nm', uci_data_dir=uci_data_dir, mode='train')
test_dataset = UCIDataset.create('pumadyn32nm', uci_data_dir=uci_data_dir, mode='test')

train_x, train_y = train_dataset.x, train_dataset.y
test_x, test_y = test_dataset.x, test_dataset.y
train_x, train_y, test_x, test_y = standardize(train_x, train_y, test_x, test_y)

### Exact GP

In [ ]:
egp = ExactGPModel(train_x, train_y).float()

train_util(egp, train_x, train_y)

### Bilateral GP

In [ ]:
bigp = BilateralGPModel(train_x, train_y).float()

with gp.settings.max_root_decomposition_size(100):
    train_util(bigp, train_x, train_y)